In [113]:
import sqlite3
import pandas as pd
import json
import hashlib
from sqlalchemy.types import VARCHAR, Float, Integer, Date, Numeric
pd.options.mode.chained_assignment = None

In [114]:
cnx = sqlite3.connect('./game.db')
ana = sqlite3.connect('./game_analysis.db')


In [115]:
users = pd.read_sql_query("SELECT * FROM users", cnx)
users.drop_duplicates(keep='last', inplace=True)


In [116]:

str2hash = "ender"

# encoding GeeksforGeeks using encode()
# then sending to md5()
result = hashlib.md5(str2hash.encode())
  
# printing the equivalent hexadecimal value.
print("The hexadecimal equivalent of hash is : ", end ="")
print(result.hexdigest())

The hexadecimal equivalent of hash is : 3fa01cb261078d3566dd9dbbbf7e3c63


In [117]:
import json

with open('cn.json') as data_file:
    data = json.load(data_file)

json_df = pd.DataFrame.from_dict(data, orient="index").reset_index()
json_df.columns = ['en', 'cn']
json_df.to_csv('./中文翻译.csv', index=False)

In [130]:
game_df = pd.read_sql_query("SELECT * FROM game_results", cnx)
game_df['createtime'] = pd.to_datetime(game_df['createtime'])
game_df.loc[:, ['game_id', 'players', 'generations', 'createtime']].to_sql('ods_main_games', if_exists='replace', con=ana, index=False)
ana.execute("create index ods_main_games_game_id_index on ods_main_games (game_id);")

In [131]:
game_df['scores'] = game_df['scores'].apply(lambda x:eval(x))

In [132]:
tmp_game_df = pd.melt(pd.json_normalize(game_df['scores']).reset_index(), id_vars='index', value_name='player_data').dropna(subset=['player_data']).set_index('index', drop=True)
flat_game_df = (pd.json_normalize(tmp_game_df['player_data']).join(tmp_game_df.reset_index(drop=False)['index']).set_index('index')) \
    .join(game_df.drop(['seed_game_id', 'game_options', 'scores'], axis=1)).reset_index()
flat_game_df['position'] = flat_game_df.sort_index() \
            .groupby(['game_id']) \
            .cumcount() + 1
flat_game_df['rank'] = flat_game_df.sort_values(['playerScore'], ascending=[False]) \
            .groupby(['game_id']) \
            .cumcount() + 1


In [133]:
flat_game_df.drop('index', axis=1).to_sql('ods_game_results', con=ana, if_exists='replace')

In [135]:
sql = """
    select corporation,
        players,
        avg(rank)                           as rank,
        avg(playerScore)                    as score,
        avg(generations)                    as generations,
        count(corporation)                  as total_games
    from (select corporation,
                playerScore,
                players,
                rank,
                generations
        from ods_game_results
        union all
        select corporation2,
                playerScore,
                players,
                rank,
                generations
        from ods_game_results
        where corporation2 <> '')
    group by corporation, players
    order by players desc, rank desc;
"""
corp_df = pd.read_sql(sql, con=ana)

In [136]:
corp_df

,corporation,players,rank,score,generations,total_games
0,Interplanetary Cinematics (breakthrough),5,5.0,49.0,6.0,1
1,Factorum (breakthrough),5,5.0,49.0,6.0,1
2,Vitor,5,4.0,65.0,6.0,1
3,Poseidon,5,4.0,65.0,6.0,1
4,🌸Protogen🌸,5,3.0,67.0,6.0,1
...,...,...,...,...,...,...
163,Helion (breakthrough),2,1.0,102.0,9.0,1
164,Factorum,2,1.0,128.0,15.0,1
165,Collegium Copernicus,2,1.0,148.0,11.0,1
166,Arklight (breakthrough),2,1.0,102.0,9.0,1


In [137]:
def get_full_games(cnx):
    res = pd.read_sql_query("SELECT * FROM games where status = 'finished' order by save_id", cnx).drop_duplicates(keep='last', inplace=False)
    return res
detail_df = get_full_games(cnx)

In [138]:
# 选择所有已完成数据
false = False
true = True
null = None
def getCardData(detail_df, game_df):
    df1a = detail_df

    #  关联快照表，取所需字段，即game_df可以通过streamlit选项动态调整
    df2 = game_df.merge(df1a,how='left',on=['game_id', 'game_id'], suffixes=['','_drop'], indicator=True).query('_merge == "both"')

    # df2 = df1b[df1b['players'] == player_number]
    # 修改需要展开字段的类型为list
    df2['game'] = df2['game'].apply(lambda x:eval(x))
    mid = pd.json_normalize(df2['game'],record_path=['players', 'playedCards'], meta=['id',  ['players','terraformRating'], ['players','name']], sep='_') \
        .drop('targetCards', axis=1).fillna('')
    mid = mid.astype(str)
    # mid['players'] = player_number
    return mid
mid = getCardData(detail_df, game_df)
# mid2 = getCardData(detail_df, game_df, 2)
# mid = pd.concat([mid2, mid4])



In [139]:
# dtypedict = {'name' : VARCHAR(30), 'resourceCount' : Integer, 'cloneTag': VARCHAR(10), 'isDisabled' : VARCHAR(5), 'bonusResource' : VARCHAR(10), 
#        'id' : VARCHAR(13), 'players_terraformRating' : Integer, 'players_name': VARCHAR(20)}
mid.to_sql('ods_card_results', con=ana, if_exists='replace')

In [143]:
ana.execute("drop table if exists dwd_cards;")
sql2 = """
    create table dwd_cards as
    select game_id,
        player,
        corporation,
        corporation2,
        name                    as card_name,
        playerScore,
        cast(players_terraformRating as INTEGER) as tr,
        players,
        generations,
        position,
        rank,
        createtime,
        cast(resourceCount as INTEGER) as resourceCount,
        cloneTag,
        isDisabled,
        bonusResource,
        userId
    from ods_card_results
            inner join ods_game_results on ods_card_results.id = ods_game_results.game_id and
                                        ods_card_results.players_name = ods_game_results.player
"""
ana.execute(sql2)

In [146]:
mid1 = mid.loc[:,['id','players_name','name','resourceCount','players_terraformRating', 'players']]
mid1['rn'] = mid1 \
            .groupby(['id','players_name', 'players']) \
            .cumcount() + 1
# mid
# TODO  需要关联处理后的game_result表，获取position, gens, createtime等数据, 只能通过merge实现

In [153]:
def getPlayerNumResult(df, player_num = 4):
    """
    主键: game_id, player
    """
    df = df.loc[df['players'] == player_num].reset_index(drop=True)
    for i in range(1, player_num+1):
        player_idx = 'player'+str(i)
        # player_df_pre = df[player_idx].apply(lambda x:eval(x))
        # print(player_idx)
        # player_df = pd.json_normalize(player_df_pre).reset_index(drop=True)
        player_df = pd.json_normalize(df[player_idx]).reset_index(drop=True)
        if i == 1:
            res = pd.concat([df,player_df.reindex(df.index)],axis=1)
            print((res.loc[pd.isna(res['player']) == False]).shape[0])
        else:
            mid = pd.concat([df,player_df.reindex(df.index)],axis=1)
            res = pd.concat([res, mid],axis=0, ignore_index=True)
            # print((mid.loc[pd.isna(mid['player']) == False]).shape[0])
        # df = pd.concat([df, pd.json_normalize(df[player_idx])],axis=1)
    res.drop(['player'+str(i) for i in range(1, 7)], axis=1, inplace=True)
    res['count'] = 1
    res['players'] = player_num
    return res

def getCardRank(df):
    """
    对card做聚合, 取打出胜率以及打出次数
    """
    # res = df.query('rn not in (1,2)') \
    res = df \
    .groupby(['name', 'players']) \
    .agg(
        position = ('position', 'mean'),
        playerScore = ('playerScore', 'mean'),
        generations = ('generations', 'mean'),
        total = ('count', 'sum')
    ) \
    .dropna().sort_values('position').reset_index()
    cn = pd.read_csv('./中文翻译.csv')
    res = res.merge(cn, left_on = ['name'], right_on = ['en'], how = 'left')
    first_column = res.pop('cn')

    res.insert(0, 'cn', first_column)
    res.loc[pd.isna(res['cn'])==True,'cn'] = res.loc[pd.isna(res['cn'])==True,'name']
    res = res.drop('en', axis = 1)
    return res

def getPlayersCardRank(df):
    """
    对card做聚合, 取打出胜率以及打出次数
    """
    res = df.query('rn not in (1,2)') \
    .groupby(['player', 'name', 'players']) \
    .agg(
        position = ('position', 'mean'),
        playerScore = ('playerScore', 'mean'),
        generations = ('generations', 'mean'),
        total = ('count', 'sum'),
        sum_position = ('position', 'sum'),
        sum_playerScore = ('playerScore', 'sum'),
        sum_generations = ('generations', 'sum')
    ) \
    .dropna().sort_values('position').reset_index()
    cn = pd.read_csv('./中文翻译.csv')
    res = res.merge(cn, left_on = ['name'], right_on = ['en'], how = 'left')
    first_column = res.pop('cn')

    res.insert(0, 'cn', first_column)
    res.loc[pd.isna(res['cn'])==True,'cn'] = res.loc[pd.isna(res['cn'])==True,'name']
    res = res.drop('en', axis = 1)
    return res

player_df4 = getPlayerNumResult(df4, 4) # 主键: game_id, player
player_df2 = getPlayerNumResult(df4, 2) # 主键: game_id, player
player_df = pd.concat([player_df4, player_df2])

card_df = mid1.merge(player_df, how='left', left_on=['id', 'players_name'], right_on=['game_id', 'player'], suffixes=['','_drop'], indicator=True).query('_merge == "both"')

card_df_group = getCardRank(card_df)
card_df.to_csv('CardDetail.csv', index=False)
getPlayersCardRank(card_df).to_csv('playersCardRank.csv', index=False)
card_df_group.sort_values('total',ascending=False).to_csv('./allCardsRank.csv', index = False)

4431
5852


In [151]:
card_df

,id,players_name,name,resourceCount,players_terraformRating,players,rn,game_id,seed_game_id,players_drop,...,escapeVelocityPenalty,corporation,playerScore,player,order,position,userId,corporation2,count,_merge
0,gbf004fccd91d,zd,Mining Area,NaN,51,2,1,gbf004fccd91d,None,2,...,NaN,Saturn Systems,121,zd,2,1,NaN,,1,both
1,gbf004fccd91d,zd,Underground Detonations,NaN,51,2,2,gbf004fccd91d,None,2,...,NaN,Saturn Systems,121,zd,2,1,NaN,,1,both
2,gbf004fccd91d,zd,Standard Technology,NaN,51,2,3,gbf004fccd91d,None,2,...,NaN,Saturn Systems,121,zd,2,1,NaN,,1,both
3,gbf004fccd91d,zd,Toll Station,NaN,51,2,4,gbf004fccd91d,None,2,...,NaN,Saturn Systems,121,zd,2,1,NaN,,1,both
4,gbf004fccd91d,zd,Acquired Company,NaN,51,2,5,gbf004fccd91d,None,2,...,NaN,Saturn Systems,121,zd,2,1,NaN,,1,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437661,g918fcce6146c,啸,Cloud Seeding,0.0,30,4,19,g918fcce6146c,None,4,...,1,Tharsis Republic,64,啸,3,3,NaN,,1,both
437662,g918fcce6146c,啸,Grass,0.0,30,4,20,g918fcce6146c,None,4,...,1,Tharsis Republic,64,啸,3,3,NaN,,1,both
437663,g918fcce6146c,啸,Ice Cap Melting,0.0,30,4,21,g918fcce6146c,None,4,...,1,Tharsis Republic,64,啸,3,3,NaN,,1,both
437664,g918fcce6146c,啸,Worms,0.0,30,4,22,g918fcce6146c,None,4,...,1,Tharsis Republic,64,啸,3,3,NaN,,1,both


In [69]:
card_df[card_df['player'] == '钝钝']

,id,players_name,name,rn,game_id,seed_game_id,players,generations,createtime,preludeExtension,...,escapeVelocityPenalty,corporation,playerScore,player,order,position,userId,corporation2,count,_merge
117,ga236647f0eb3,钝钝,UNMI Contractor,1,ga236647f0eb3,None,4,6,2021-06-15 13:42:15,True,...,NaN,UNMI (breakthrough),83,钝钝,2,1,69215eb418a5,Aphrodite (breakthrough),1,both
118,ga236647f0eb3,钝钝,Metal-Rich Asteroid,2,ga236647f0eb3,None,4,6,2021-06-15 13:42:15,True,...,NaN,UNMI (breakthrough),83,钝钝,2,1,69215eb418a5,Aphrodite (breakthrough),1,both
119,ga236647f0eb3,钝钝,Soletta,3,ga236647f0eb3,None,4,6,2021-06-15 13:42:15,True,...,NaN,UNMI (breakthrough),83,钝钝,2,1,69215eb418a5,Aphrodite (breakthrough),1,both
120,ga236647f0eb3,钝钝,Olympus Conference,4,ga236647f0eb3,None,4,6,2021-06-15 13:42:15,True,...,NaN,UNMI (breakthrough),83,钝钝,2,1,69215eb418a5,Aphrodite (breakthrough),1,both
121,ga236647f0eb3,钝钝,Optimal Aerobraking,5,ga236647f0eb3,None,4,6,2021-06-15 13:42:15,True,...,NaN,UNMI (breakthrough),83,钝钝,2,1,69215eb418a5,Aphrodite (breakthrough),1,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168154,gc2ac3fd5df38,钝钝,Herbivores,35,gc2ac3fd5df38,None,4,6,2022-02-18 18:02:26,True,...,1,Splice (breakthrough),68,钝钝,2,3,69215eb418a5,Celestic (breakthrough),1,both
168155,gc2ac3fd5df38,钝钝,Heather,36,gc2ac3fd5df38,None,4,6,2022-02-18 18:02:26,True,...,1,Splice (breakthrough),68,钝钝,2,3,69215eb418a5,Celestic (breakthrough),1,both
168156,gc2ac3fd5df38,钝钝,Advanced Ecosystems,37,gc2ac3fd5df38,None,4,6,2022-02-18 18:02:26,True,...,1,Splice (breakthrough),68,钝钝,2,3,69215eb418a5,Celestic (breakthrough),1,both
168157,gc2ac3fd5df38,钝钝,Trans-Neptune Probe,38,gc2ac3fd5df38,None,4,6,2022-02-18 18:02:26,True,...,1,Splice (breakthrough),68,钝钝,2,3,69215eb418a5,Celestic (breakthrough),1,both


In [43]:
def getPlayerCardRank(df, player):
    ff1 = df.query('player == "%s"'%(player))
    return getCardRank(ff1)

In [74]:
card_rank = getPlayersCardRank(card_df)
card_rank[card_rank['player'] == 'ender']

,player,name,position,playerScore,generations,total,sum_position,sum_playerScore,sum_generations
2707,ender,Great Dam:promo,1.0,86.0,6.0,1,1,86,6
2710,ender,Floater-Urbanism,1.0,86.0,6.0,1,1,86,6
2713,ender,Diaspora Movement,1.0,86.0,6.0,1,1,86,6
2717,ender,Birds,1.0,86.0,6.0,1,1,86,6
2718,ender,Atmoscoop,1.0,86.0,6.0,1,1,86,6
...,...,...,...,...,...,...,...,...,...
89593,ender,Heather,4.0,85.0,6.0,1,4,85,6
89594,ender,Geological Expedition,4.0,85.0,6.0,1,4,85,6
89595,ender,GMO Contract,4.0,85.0,6.0,1,4,85,6
89596,ender,Floater Leasing,4.0,85.0,6.0,1,4,85,6


In [80]:
card_df_group.sort_values('total',ascending=False).to_csv('./allCardsRank.csv', index = False)


In [97]:
false = False
true = True
player_df_new = df2['game'].apply(lambda x:eval(x))

In [143]:
player_df_new

6950    {'id': 'gb53baf3212e3', 'players': [{'name': '...
7020    {'id': 'g32498849988e', 'players': [{'name': '...
7068    {'id': 'gd4c2a4f2bbf3', 'players': [{'name': '...
7083    {'id': 'gb53baf3212e3', 'players': [{'name': '...
7089    {'id': 'g3855ccb42bec', 'players': [{'name': '...
Name: game, dtype: object

In [156]:
# method1
mid = pd.json_normalize(player_df_new,record_path=['players', 'playedCards'], meta=[['players','terraformRating'], ['players','name']], sep='_')
mid
# pd.json_normalize(mid.iloc[0],record_path=['playedCards'])


,name,resourceCount,bonusResource,players_terraformRating,players_name
0,Biolab,NaN,NaN,33,米饭
1,Aerospace Mission,NaN,NaN,33,米饭
2,Nitrate Reducers,NaN,NaN,33,米饭
3,Psychrophiles,1.0,NaN,33,米饭
4,Invention Contest,NaN,NaN,33,米饭
...,...,...,...,...,...
115,Underground City,NaN,NaN,56,You
116,Algae,NaN,NaN,56,You
117,Sub-Crust Measurements,NaN,NaN,56,You
118,Galilean Waystation,NaN,NaN,56,You


In [120]:
# pd.json_normalize(player_df_new,record_path=['players'],meta=['id'])

# mid = pd.json_normalize(player_df_new,record_path=['players'])
mid = pd.json_normalize(player_df_new)['players']
# pd.json_normalize(mid['players'],record_path=['playedCards'],meta=['name',['_game','id']], record_prefix='*')
# pd.json_normalize(mid['players'],record_path=['playedCards'])

In [133]:
mid

0    [{'name': 'You', 'color': 'red', 'beginner': F...
1    [{'name': '米饭', 'color': 'red', 'beginner': Fa...
2    [{'name': 'You', 'color': 'red', 'beginner': F...
3    [{'name': 'You', 'color': 'red', 'beginner': F...
4    [{'name': 'You', 'color': 'red', 'beginner': F...
Name: players, dtype: object

In [134]:
mid = mid.tail(1)

In [139]:
mid.iloc[0]

[{'name': 'You',
  'color': 'red',
  'beginner': False,
  'handicap': 0,
  '_game': {'id': 'g3855ccb42bec'},
  'corpCard': {'name': 'Saturn Systems'},
  'pickedCorporationCard': {'name': 'Saturn Systems'},
  'terraformRating': 56,
  'hasIncreasedTerraformRatingThisGeneration': True,
  'terraformRatingAtGenerationStart': 14,
  'megaCredits': 63,
  'megaCreditProduction': 7,
  'steel': 4,
  'steelProduction': 4,
  'titanium': 1,
  'titaniumProduction': 1,
  'plants': 3,
  'plantProduction': 11,
  'energy': 3,
  'energyProduction': 3,
  'heat': 15,
  'heatProduction': 13,
  'titaniumValue': 3,
  'steelValue': 2,
  'canUseHeatAsMegaCredits': False,
  'actionsTakenThisRound': 0,
  'actionsThisGeneration': [],
  'undoing': False,
  'exited': False,
  'canExit': False,
  'corpInitialActionDone': False,
  'corp2InitialActionDone': False,
  'dealtCorporationCards': [],
  'dealtProjectCards': [],
  'dealtPreludeCards': [],
  'cardsInHand': [{'resourceCount': 0, 'name': 'Martian Zoo'},
   {'resou

In [140]:
pd.json_normalize(mid.iloc[0])

,name,color,beginner,handicap,terraformRating,hasIncreasedTerraformRatingThisGeneration,terraformRatingAtGenerationStart,megaCredits,megaCreditProduction,steel,...,heatProductionStepsIncreasedThisGeneration,id,_game.id,corpCard.name,pickedCorporationCard.name,timer.sumElapsed,timer.startedAt,timer.running,timer.afterFirstAction,timer.lastStoppedAt
0,You,red,False,0,56,True,14,63,7,4,...,0,p78988c7af87e,g3855ccb42bec,Saturn Systems,Saturn Systems,1430484,1627868526517,False,True,1627868527001


In [141]:
pd.json_normalize(mid.iloc[0],record_path=['playedCards'])

,name,resourceCount,bonusResource
0,Mohole Excavation,NaN,NaN
1,Business Empire,NaN,NaN
2,Regolith Eaters,0.0,NaN
3,Convoy From Europa,NaN,NaN
4,Subterranean Reservoir,NaN,NaN
5,Lichen,NaN,NaN
6,Mohole Area,NaN,NaN
7,Trade Envoys,NaN,NaN
8,Deep Well Heating,NaN,NaN
9,Peroxide Power,NaN,NaN


In [64]:
false = False
true = True
player_df_pre = df2['game'].apply(lambda x:eval(x))
# print(player_idx)
player_df = pd.json_normalize(player_df_pre).reset_index(drop=True)
player_df = player_df.loc[:,['id', 'players']]
# pd.DataFrame([flatten_json(x) for x in eval(df.iloc[0,2])])

In [67]:
# player_df['players'] = player_df['players'].apply(lambda x:eval(x))
player_df2 = player_df.explode('players')
# player_df3a = player_df2['players'].apply(lambda x:eval(x))
# print(player_idx)
player_df3 = pd.json_normalize(player_df2).reset_index(drop=True)

In [91]:
# player_df2['players'] = eval(player_df2['players'])
player_df2['players']


0    {'name': 'You', 'color': 'red', 'beginner': Fa...
1    {'name': '米饭', 'color': 'red', 'beginner': Fal...
1    {'name': '小麦', 'color': 'green', 'beginner': F...
2    {'name': 'You', 'color': 'red', 'beginner': Fa...
3    {'name': 'You', 'color': 'red', 'beginner': Fa...
4    {'name': 'You', 'color': 'red', 'beginner': Fa...
Name: players, dtype: object

In [89]:
player_df2['players'] = player_df2['players'].astype(str).apply(lambda x:eval(x))

In [122]:
f = open('./count.txt')
count = f.read()
f.close()
f = open('./count.txt', 'w')
add = str(int(count)+1)
f.write(add)
f.close()


In [93]:
player_df3 = pd.json_normalize(player_df2['players'])[:,'']

,name,color,beginner,handicap,terraformRating,hasIncreasedTerraformRatingThisGeneration,terraformRatingAtGenerationStart,megaCredits,megaCreditProduction,steel,...,waitingFor.options,userId,corpCard.name,pickedCorporationCard.name,corpCard.tags,corpCard.startingMegaCredits,corpCard.cardType,pickedCorporationCard.tags,pickedCorporationCard.startingMegaCredits,pickedCorporationCard.cardType
0,You,red,False,0,14,False,14,0,0,0,...,"[{'title': 'Select corporation', 'buttonLabel'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,米饭,red,False,0,33,True,20,58,21,1,...,NaN,d484c04d1ffb,Valley Trust (breakthrough),Valley Trust (breakthrough),NaN,NaN,NaN,NaN,NaN,NaN
2,小麦,green,False,0,52,True,20,71,15,0,...,NaN,dfc7613bd132,Interplanetary Cinematics (breakthrough),Interplanetary Cinematics (breakthrough),[building],50.0,corporation,[building],50.0,corporation
3,You,red,False,0,62,True,14,82,7,12,...,NaN,NaN,EcoLine,EcoLine,NaN,NaN,NaN,NaN,NaN,NaN
4,You,red,False,0,37,True,14,56,11,12,...,NaN,NaN,CrediCor,CrediCor,NaN,NaN,NaN,NaN,NaN,NaN
5,You,red,False,0,56,True,14,63,7,4,...,NaN,NaN,Saturn Systems,Saturn Systems,NaN,NaN,NaN,NaN,NaN,NaN
